In [9]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from collections import defaultdict
import torch
import pickle
from re import sub
import torch
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
from tqdm import tqdm
import torch.nn as nn
import time
from custom_transformer import CustomTransformer
import torch.nn.functional as F
import random

In [ ]:
class TextLoader:
    def __init__(self, PATH) -> None:
        
        self.corpus = open(PATH, 'r').read()
        self.sentences = self.corpus.split('\n')

    def remove_rare_words(self,vocab_dict, min_repeat):

        wordList = [k for k,v in vocab_dict.items() if v >= min_repeat]
        vocab = set(wordList)
        print(len(vocab), len(wordList))

    
    def get_all_chars(self):
        chars = defaultdict(lambda: 0)
        for sen in self.sentences:
            for char in sen:
                chars[char] += 1

        return chars

    def remove_sentence_with_rare_chars(self, min_count, dict):
        sens_idx = []
        for idx, sen in enumerate(self.sentences):
            flag = 1
            for char in sen:
                if(dict[char] < min_count):
                    flag = 0
                    break
            if flag:
                sens_idx.append(idx)
        return sens_idx

    def create_cleaned_data(self, idxs, ext):
        cleaned_sentences = [self.sentences[i] for i in idxs]
        self.sentences = cleaned_sentences
        print(len(self.sentences))

        clean_file = open('cleaned_train.' + ext, 'w+')
        for i in range(len(self.sentences)):  
            clean_file.write(self.sentences[i] + '\n')
            
    def batch_iterator(self, batch_size):
        for i in range(0, len(self.sentences), batch_size):
            yield self.sentences[i : i + batch_size]


In [2]:
# loader_en = TextLoader('train.en')
# loader_de = TextLoader('train.de')

In [3]:
# chars_en = loader_en.get_all_chars()
# chars_de = loader_de.get_all_chars()

# chars = defaultdict(lambda:0)
# for k,v in chars_en.items():
#     chars[k] += v
# for k,v in chars_de.items():
#     chars[k] += v

# chars = dict(sorted(chars.items(), key=lambda item: item[1], reverse=True))
# # dict(sorted(chars_de.items(), key=lambda item: item[1], reverse=True))

# cleaned_en = loader_en.remove_sentence_with_rare_chars(1000, chars)
# cleaned_de = loader_de.remove_sentence_with_rare_chars(1000, chars)
# cleaned_idx = list(set(cleaned_en) & set(cleaned_de))
# len(cleaned_idx), len(cleaned_en), len(cleaned_de)

# loader_de.create_cleaned_data(cleaned_idx, 'de')
# loader_en.create_cleaned_data(cleaned_idx, 'en')


(4442493, 4453061, 4449785)

In [2]:
loader_en = TextLoader('cleaned_train.en')
loader_de = TextLoader('cleaned_train.de')

assert(len(loader_de.sentences) == len(loader_en.sentences))

In [ ]:
tokenizer_en = Tokenizer(models.BPE())
tokenizer_en.normalizer = normalizers.Lowercase()
# tokenizer_en.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

tokenizer_en.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

trainer_en = trainers.BpeTrainer(vocab_size=25000,special_tokens=["[PAD]"])
tokenizer_en.train_from_iterator(loader_en.batch_iterator(1000), trainer=trainer_en)

# tokenizer_en.decoder =  decoders.ByteLevel()
tokenizer_en.save("tokenizer_en_25000_start_token_SOS.json")

In [16]:
tokenizer_en.decode([527, 1263, 197, 627, 48, 0, 0, 0])

'my name is bas u'

In [15]:
encoding = tokenizer_en.encode("My name is Basu")
print(encoding.tokens,encoding.ids )
tokenizer_en.decode(encoding.ids)

for batch in loader_en.batch_iterator(2):
    encoding = tokenizer_en.encode_batch(batch)
    print(tokenizer_en.decode_batch([encoding[i].ids for i in range(len(encoding))]))
    break

['my', 'name', 'is', 'bas', 'u'] [527, 1263, 197, 627, 48]


'my name is bas u'

In [ ]:
li = []
for batch in loader_en.batch_iterator(1024):
    encoding = tokenizer_en.encode_batch(batch)
    for i in range(len(encoding)):
        li.append(len(encoding[i].ids))

li = sorted(li, reverse=True)

In [ ]:
for i in range(5000, 6000):
    if li[i] <= 100:
        print(i)

import matplotlib.pyplot as plt
plt.hist(li)

In [ ]:
tokenizer_de = Tokenizer(models.BPE())
tokenizer_de.normalizer = normalizers.Lowercase()
# tokenizer_de.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

tokenizer_de.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

# print(tokenizer.pre_tokenizer.pre_tokenize_str("Ich liebe das wirklich ."))

trainer_de = trainers.BpeTrainer(vocab_size=25000, special_tokens=["[PAD]", "[SOS]", "[EOS]"])
tokenizer_de.train_from_iterator(loader_de.batch_iterator(1000), trainer=trainer_de)

tokenizer_de.post_processor = processors.TemplateProcessing(
    single=f"[SOS]:0 $A:0 [EOS]:0",
    special_tokens=[("[SOS]", 0), ("[EOS]", 1)])
# tokenizer_de.decoder =  decoders.ByteLevel()

tokenizer_de.save("tokenizer_de_25000_start_token_SOS.json")

In [ ]:
tokenizer_de.decode([i for i in range(300)])

In [20]:
encoding = tokenizer_de.encode("My name is Basu")
print(encoding.tokens, encoding.ids)
tokenizer_de.decode(encoding.ids + [0,0,0,0])

['[SOS]', 'my', 'name', 'is', 'bas', 'u', '[EOS]'] [0, 2985, 3725, 250, 1171, 50, 1]


'my name is bas u'

In [14]:
# for batch in loader_de.batch_iterator(1024):
#     encoding = tokenizer_de.encode_batch(batch)
#     a = tokenizer_de.decode_batch([encoding[i].ids for i in range(len(encoding))])

In [4]:
tokenizer_de = Tokenizer.from_file("tokenizer_de_25000_start_token_SOS.json")
tokenizer_en = Tokenizer.from_file("tokenizer_en_25000_start_token_SOS.json")

In [22]:
en_encodes = []
encodes_len = []
for idx, batch in enumerate(loader_en.batch_iterator(1024)):
    encoding = tokenizer_en.encode_batch(batch)
    for i in range(len(encoding)):
        en_encodes.append(encoding[i].ids)
        encodes_len.append(len(encoding[i].ids))

de_encodes = []
for idx, batch in enumerate(loader_de.batch_iterator(1024)):
    encoding = tokenizer_de.encode_batch(batch)
    for i in range(len(encoding)):
        de_encodes.append(encoding[i].ids)

encodes_len , en_encodes, de_encodes = (list(t) for t in zip(*sorted(zip(encodes_len , en_encodes, de_encodes))))


# Dump the tokenized lists as pickle files for faster retrieval
with open('en_tokenized.pkl', 'wb') as f:
    pickle.dump(en_encodes, f)

with open('de_tokenized.pkl', 'wb') as f:
    pickle.dump(de_encodes, f)

In [15]:
with open('en_tokenized.pkl', 'rb') as f:
    en_tokens = pickle.load(f)

with open('de_tokenized.pkl', 'rb') as f:
    de_tokens = pickle.load(f)

assert(len(de_tokens) == len(en_tokens))

In [12]:
def loader(en_tokens, de_tokens, bs, pad_idx, shuffle = True):
    num_batches = len(en_tokens)//bs
    idxs = [i for i in range(num_batches)]

    if shuffle:
        random.shuffle(idxs)

    for i in idxs:
        max_len = len(en_tokens[(i+1)*bs - 1])
        en_tensor = torch.tensor([enc + 
        [pad_idx]*(max_len - len(enc)) for enc in en_tokens[i*bs:(i+1)*bs]])
        
        en_pad_idx = torch.tensor((en_tensor != pad_idx)).unsqueeze(1).unsqueeze(2)

        max_len= max([len(enc) for enc in de_tokens[i*bs:(i+1)*bs]])
        de_tensor = torch.tensor([enc + 
        [pad_idx]*(max_len - len(enc)) for enc in de_tokens[i*bs:(i+1)*bs]])

        de_mask = torch.ones(bs, max_len, max_len).tril().unsqueeze(1)


        yield en_tensor, en_pad_idx, de_tensor, de_mask

In [7]:
nx = 6
edim = 512
hdim = 2048
h = 8
nwords = 8
src_vocab_size = 25000
tgt_vocab_size = 25000
n_epochs = 10
bs = 32
pad_idx = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# src_embed = torch.rand(bs, nwords, edim).to(device)
# tgt_embed = torch.rand(bs, nwords, edim).to(device)
# probs = transformer(src_embed, tgt_embed)
# print(probs.shape)

net = CustomTransformer(nx, edim, h, hdim, src_vocab_size, tgt_vocab_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), betas=[0.9, 0.98])

cuda


In [14]:
ep = 0
for epoch in range(n_epochs):  # loop over the dataset multiple times

    correct, total = 0, 0
    for src, src_mask, tgt, tgt_mask in loader(en_tokens, de_tokens, bs, pad_idx):
        ep+=1
        src = src.to(device)
        src_mask = src_mask.to(device)
        tgt = tgt.to(device)
        tgt_mask = tgt_mask.to(device)

        optimizer.zero_grad()

        outputs = net(src, tgt, src_mask, tgt_mask)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted.shape)
        break
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if ep % num_steps == 0: 
#             val_accu = val(valloader)
#             print("epoch {0} | loss: {1:.2f} | Val Accuracy: {2:.2f} %".format(epoch, running_loss/num_steps, val_accu ))
#             running_loss = 0.0
#             if val_accu > best_accu:
#               best_accu= val_accu 
#               torch.save(net.state_dict(), 'net_val.pth')
#               print("Saving")
#             net.train()
#     print("Train accuracy: {0:.2f} %".format(100 * correct / total))

# print('Finished Training')

NameError: name 'en_tokens' is not defined

In [ ]:
# from transformers import PreTrainedTokenizerFast

# fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer_de_25000_start_token_SOS.json")

# for batch in loader_de.batch_iterator(1024):
#     # print(batch)
#     encoding = fast_tokenizer(batch)
#     # decoded = fast_tokenizer.batch_decode(encoding['input_ids'])
#     # print(decoded)
#     # break